### Limitations of Spheros

- Heading gets messed up when it bumps into things
- Very limited sensors
    - Gyro
    - Accelerometer
    
- Unusable location sensors
    - bot.read_locator()
        - x,y position
        - 'sog' (speed on ground)
        
- Limited gripping friction (smooth surface) robot shell means it cannot push from a standstill effectively unless the target weighs significantly less than the sphero (physics)




### Videos

- Embed here


### Proposed Strategy

Based on testing with the blocks on 4/22 to overcome above limitations, I believe that the optimal pushing strategy looks something like this. 


Note that we work with objects with low surface friction, i.e. putting wooden blocks on top of a plastic tray.

- Camera detects "pushing plane" of the target: the line on the object that the robots should be pushing against. The pushing plane is perpendicular to a direct line between the object and the target destination.

- Robots move into a formation so that they are symmetrically about a line perpendicular to the "pushing plane"
- Robots then roll rapidly towards the pushing plane. Camera begins tracking them.

At this point, we know there are significant latency issues when more than 3 robots are being tracked at once. I propose the following modification to the traditional full tracking algorithm.


    - On each frame, track the camera and just 1 of the robots. (Note the camera captures 20 FPS). Take turns re: which robot is being tracked.
    
    - If a robot is behind the pushing plane, let it drive at constant speed towards the target, and update its pushing heading. (Only use PID controller if we are worried about the robots pushing the box too far/ with inertia). Status is "PUSHING"
    - If robot is ahead of the pushing plane, set status to "RETREATING".


After each frame has been processed, we then send a roll command to each robot based on its status.


    - If status is "PUSHING", push it along the current heading
    - If status is "RETREATING", somehow get robot away from other robots, and pilot it back towards some fixed distance behind the pushing plane. (Maybe at 45 degree angles or something? They might bump towards each other.
    - Break when object has reached its target destination.
    